In [ ]:
# import tensorflow as tf
# from sklearn.model_selection import train_test_split

# # Define the dataset directory
# dataset_dir = "./PlantVillage-Dataset/raw/color"

# # Load the dataset and split it into training and validation sets
# train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
#     dataset_dir,
#     validation_split=0.2,  # Reserve 20% of the data for validation
#     subset="training",
#     seed=123,  # Ensures reproducibility
#     image_size=(224, 224),  # Resize images to this size
#     batch_size=32,  # Adjust based on your needs
#     label_mode='categorical'  # Use 'categorical' for multi-class labels
# )

# val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
#     dataset_dir,
#     validation_split=0.2,  # Reserve 20% of the data for validation
#     subset="validation",
#     seed=123,  # Ensures reproducibility
#     image_size=(224, 224),
#     batch_size=32,
#     label_mode='categorical'
# )

# # Further split the validation dataset into validation and test sets
# val_batches = tf.data.experimental.cardinality(val_dataset)
# test_dataset = val_dataset.take(val_batches // 2)
# val_dataset = val_dataset.skip(val_batches // 2)

# # Configure the dataset for performance
# AUTOTUNE = tf.data.AUTOTUNE
# train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
# val_dataset = val_dataset.prefetch(buffer_size=AUTOTUNE)
# test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

### Moving the files around

In [13]:
import os
import shutil
from sklearn.model_selection import train_test_split
import tensorflow as tf

original_dataset_dir = './PlantVillage-Dataset/raw/color'
new_base_dir = './PlantVillage-Dataset-processed'

# Splitting ratio
train_ratio = 0.8
test_ratio = 0.2

# Ensure the new directory exists
os.makedirs(new_base_dir, exist_ok=True)

In [14]:
# Function to resize and save an image
def resize_and_save_image(src_path, dest_path, new_size=(256, 256)):
    image = tf.keras.preprocessing.image.load_img(src_path, target_size=new_size)
    image.save(dest_path)

# Loop through each class directory
for class_name in os.listdir(original_dataset_dir):
    print(f'Processing {class_name} ...')
    # Create new directories for train and test sets for the current class
    os.makedirs(os.path.join(new_base_dir, 'train', class_name), exist_ok=True)
    os.makedirs(os.path.join(new_base_dir, 'test', class_name), exist_ok=True)

    # List all images in the current class directory
    images = os.listdir(os.path.join(original_dataset_dir, class_name))
    images = [img for img in images if img.lower().endswith(('png', 'jpg', 'jpeg'))]  # Filter out non-image files

    # Split images into train and test sets
    train_images, test_images = train_test_split(images, test_size=test_ratio, random_state=42)

    # Resize and save train images
    for img in train_images:
        src_path = os.path.join(original_dataset_dir, class_name, img)
        dest_path = os.path.join(new_base_dir, 'train', class_name, img)
        resize_and_save_image(src_path, dest_path)

    # Resize and save test images
    for img in test_images:
        src_path = os.path.join(original_dataset_dir, class_name, img)
        dest_path = os.path.join(new_base_dir, 'test', class_name, img)
        resize_and_save_image(src_path, dest_path)


Processing Apple___Apple_scab ...
Processing Apple___Black_rot ...
Processing Apple___Cedar_apple_rust ...
Processing Apple___healthy ...
Processing Blueberry___healthy ...
Processing Cherry_(including_sour)___healthy ...
Processing Cherry_(including_sour)___Powdery_mildew ...
Processing Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot ...
Processing Corn_(maize)___Common_rust_ ...
Processing Corn_(maize)___healthy ...
Processing Corn_(maize)___Northern_Leaf_Blight ...
Processing Grape___Black_rot ...
Processing Grape___Esca_(Black_Measles) ...
Processing Grape___healthy ...
Processing Grape___Leaf_blight_(Isariopsis_Leaf_Spot) ...
Processing Orange___Haunglongbing_(Citrus_greening) ...
Processing Peach___Bacterial_spot ...
Processing Peach___healthy ...
Processing Pepper,_bell___Bacterial_spot ...
Processing Pepper,_bell___healthy ...
Processing Potato___Early_blight ...
Processing Potato___healthy ...
Processing Potato___Late_blight ...
Processing Raspberry___healthy ...
Processing